In [1]:
import os
import numpy as np
import pandas as pd
home_folder = os.path.expanduser("~")
data_folder = os.path.join(home_folder, "Data", "basketball")
data_filename = os.path.join(data_folder, "leagues_NBA_2014_games_games.csv")

In [4]:
results = pd.read_csv(data_filename)
results.ix[:5]

,Date,Start (ET),Unnamed: 2,Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 7,Notes
0,October,October,October,October,October,October,October,October,October
1,Tue Oct 29 2013,7:00 pm,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
2,Tue Oct 29 2013,10:30 pm,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
3,Tue Oct 29 2013,8:00 pm,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
4,Wed Oct 30 2013,7:00 pm,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
5,Wed Oct 30 2013,8:30 pm,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN


In [6]:
# Don't read the first row, as it is blank, and parse the date column as a date
results = pd.read_csv(data_filename, parse_dates=["Date"], skiprows=[0,])
# Fix the name of the columns
results.columns = ["Date","Time", "Score Type", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Notes"]

results.ix[:5]

,Date,Time,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes
0,October,October,October,October,October,October,October,October,October
1,Tue Oct 29 2013,7:00 pm,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN
2,Tue Oct 29 2013,10:30 pm,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN
3,Tue Oct 29 2013,8:00 pm,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN
4,Wed Oct 30 2013,7:00 pm,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN
5,Wed Oct 30 2013,8:30 pm,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN


In [7]:
results["HomeWin"] = results["VisitorPts"] < results["HomePts"]
# Our "class values"
y_true = results["HomeWin"].values
results.ix[:5]

,Date,Time,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin
0,October,October,October,October,October,October,October,October,October,False
1,Tue Oct 29 2013,7:00 pm,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True
2,Tue Oct 29 2013,10:30 pm,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True
3,Tue Oct 29 2013,8:00 pm,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,False
4,Wed Oct 30 2013,7:00 pm,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True
5,Wed Oct 30 2013,8:30 pm,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True


In [8]:
print("Home Win percentage: {0:.1f}%".format(100 * results["HomeWin"].sum() / results["HomeWin"].count()))

Home Win percentage: 47.9%


In [9]:
results["HomeLastWin"] = False
results["VisitorLastWin"] = False
# This creates two new columns, all set to False
results.ix[:5]

,Date,Time,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
0,October,October,October,October,October,October,October,October,October,False,False,False
1,Tue Oct 29 2013,7:00 pm,Box Score,Orlando Magic,87,Indiana Pacers,97,NaN,NaN,True,False,False
2,Tue Oct 29 2013,10:30 pm,Box Score,Los Angeles Clippers,103,Los Angeles Lakers,116,NaN,NaN,True,False,False
3,Tue Oct 29 2013,8:00 pm,Box Score,Chicago Bulls,95,Miami Heat,107,NaN,NaN,False,False,False
4,Wed Oct 30 2013,7:00 pm,Box Score,Brooklyn Nets,94,Cleveland Cavaliers,98,NaN,NaN,True,False,False
5,Wed Oct 30 2013,8:30 pm,Box Score,Atlanta Hawks,109,Dallas Mavericks,118,NaN,NaN,True,False,False


In [10]:
# Now compute the actual values for these
# Did the home and visitor teams win their last game?
from collections import defaultdict
won_last = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    row["VisitorLastWin"] = won_last[visitor_team]
    results.ix[index] = row    
    # Set current win
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]
results.ix[20:25]

,Date,Time,Score Type,Visitor Team,VisitorPts,Home Team,HomePts,OT?,Notes,HomeWin,HomeLastWin,VisitorLastWin
20,November,November,November,November,November,November,November,November,November,False,False,False
21,Fri Nov 1 2013,7:30 pm,Box Score,Toronto Raptors,95,Atlanta Hawks,102,NaN,NaN,False,False,True
22,Fri Nov 1 2013,7:30 pm,Box Score,Milwaukee Bucks,105,Boston Celtics,98,NaN,NaN,True,False,False
23,Fri Nov 1 2013,8:00 pm,Box Score,Miami Heat,100,Brooklyn Nets,101,NaN,NaN,True,False,False
24,Fri Nov 1 2013,7:00 pm,Box Score,Cleveland Cavaliers,84,Charlotte Bobcats,90,NaN,NaN,True,False,True
25,Fri Nov 1 2013,9:00 pm,Box Score,Portland Trail Blazers,113,Denver Nuggets,98,NaN,NaN,True,False,True


In [11]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

In [12]:
from sklearn.cross_validation import cross_val_score

# Create a dataset with just the neccessary information
X_previouswins = results[["HomeLastWin", "VisitorLastWin"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_previouswins, y_true, scoring='accuracy')
print("Using just the last result from the home and visitor teams")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using just the last result from the home and visitor teams
Accuracy: 50.6%


In [13]:
# What about win streaks?
results["HomeWinStreak"] = 0
results["VisitorWinStreak"] = 0
# Did the home and visitor teams win their last game?
from collections import defaultdict
win_streak = defaultdict(int)

for index, row in results.iterrows():  # Note that this is not efficient
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeWinStreak"] = win_streak[home_team]
    row["VisitorWinStreak"] = win_streak[visitor_team]
    results.ix[index] = row    
    # Set current win
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [14]:
clf = DecisionTreeClassifier(random_state=14)
X_winstreak =  results[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values
scores = cross_val_score(clf, X_winstreak, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

Using whether the home team is ranked higher
Accuracy: 50.0%


In [15]:
# Let's try see which team is better on the ladder. Using the previous year's ladder
ladder_filename = os.path.join(data_folder, "leagues_NBA_2013_standings_expanded-standings.csv")
ladder = pd.read_csv(ladder_filename, skiprows=[0,1])
ladder

OSError: File b'/home/matt/Data/basketball/leagues_NBA_2013_standings_expanded-standings.csv' does not exist

In [ ]:
# We can create a new feature -- HomeTeamRanksHigher\
results["HomeTeamRanksHigher"] = 0
for index, row in results.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    if home_team == "New Orleans Pelicans":
        home_team = "New Orleans Hornets"
    elif visitor_team == "New Orleans Pelicans":
        visitor_team = "New Orleans Hornets"
    home_rank = ladder[ladder["Team"] == home_team]["Rk"].values[0]
    visitor_rank = ladder[ladder["Team"] == visitor_team]["Rk"].values[0]
    row["HomeTeamRanksHigher"] = int(home_rank > visitor_rank)
    results.ix[index] = row
results[:5]

In [ ]:
X_homehigher =  results[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_homehigher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

In [ ]:
from sklearn.grid_search import GridSearchCV

parameter_space = {
                   "max_depth": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
                   }
clf = DecisionTreeClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_homehigher, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))

In [ ]:
# Who won the last match? We ignore home/visitor for this bit
last_match_winner = defaultdict(int)
results["HomeTeamWonLast"] = 0

for index, row in results.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    teams = tuple(sorted([home_team, visitor_team]))  # Sort for a consistent ordering
    # Set in the row, who won the last encounter
    row["HomeTeamWonLast"] = 1 if last_match_winner[teams] == row["Home Team"] else 0
    results.ix[index] = row
    # Who won this one?
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner
results.ix[:5]

In [ ]:
X_home_higher =  results[["HomeTeamRanksHigher", "HomeTeamWonLast"]].values
clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_home_higher, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))


In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
encoding = LabelEncoder()
encoding.fit(results["Home Team"].values)
home_teams = encoding.transform(results["Home Team"].values)
visitor_teams = encoding.transform(results["Visitor Team"].values)
X_teams = np.vstack([home_teams, visitor_teams]).T

onehot = OneHotEncoder()
X_teams = onehot.fit_transform(X_teams).todense()

clf = DecisionTreeClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_teams, y_true, scoring='accuracy')
print("Using full team labels is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

In [ ]:
X_all = np.hstack([X_home_higher, X_teams])
print(X_all.shape)

In [ ]:
clf = RandomForestClassifier(random_state=14)
scores = cross_val_score(clf, X_all, y_true, scoring='accuracy')
print("Using whether the home team is ranked higher")
print("Accuracy: {0:.1f}%".format(np.mean(scores) * 100))

In [ ]:
#n_estimators=10, criterion='gini', max_depth=None, 
#min_samples_split=2, min_samples_leaf=1,
#max_features='auto',
#max_leaf_nodes=None, bootstrap=True,
#oob_score=False, n_jobs=1,
#random_state=None, verbose=0, min_density=None, compute_importances=None
parameter_space = {
                   "max_features": [2, 10, 'auto'],
                   "n_estimators": [100,],
                   "criterion": ["gini", "entropy"],
                   "min_samples_leaf": [2, 4, 6],
                   }
clf = RandomForestClassifier(random_state=14)
grid = GridSearchCV(clf, parameter_space)
grid.fit(X_all, y_true)
print("Accuracy: {0:.1f}%".format(grid.best_score_ * 100))
print(grid.best_estimator_)